In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from scipy.stats import zscore
from IPython.display import display

In [37]:
# df = pd.read_csv(Path('../data/2023basketballsheet.csv'), index_col=1)[lambda df_: df_["R#"] < 120]
df = (
    pd.read_csv(Path('../catsketball/staticdata/2023hashtagbasketballprojections.csv'), index_col=2)
    [lambda df_: df_["R#"] < 120]
    .sort_values("ADP")
    .assign(pick=lambda df_: df_["ADP"].rank())
    # .sort_values("R#")
    # .assign(pick=lambda df_: df_["R#"].rank())
)

In [38]:
STAT_COLS = ["FG%", "FT%", "3pm", "PTS", "TREB", "AST", "STL", "BLK", "TO"]
POSITIONS = ["PG", "SG", "SF", "PF", "C"]

In [39]:
def format_percentages(val):
    if "(" in val:
        return float(val[0:val.index('(')])
    else:
        return float(val)
    
def encode_positions(val):
    positions = {
        pos: False
        for pos in POSITIONS
    }
    for code in val.split(","):
        positions[code] = True
    return pd.Series(positions)

def clean_df(df):
    df["FG%"] = df["FG%"].apply(format_percentages)
    df["FT%"] = df["FT%"].apply(format_percentages)   
    df["TO"] = -1 * df["TO"]
    positions = df["POS"].apply(encode_positions)
    df = df.merge(positions, left_index=True, right_index=True)
    return df

df = df.pipe(clean_df)

In [40]:
df[[
    "ADP", "pick", "TEAM", "GP", "POS", *STAT_COLS
]].head(24)

,ADP,pick,TEAM,GP,POS,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
PLAYER,,,,,,,,,,,,,,
Nikola Jokic,1.1,1.0,DEN,72,C,0.624,0.821,1.0,24.5,11.7,9.7,1.4,0.7,-3.7
Joel Embiid,2.9,2.5,PHI,67,C,0.537,0.836,1.2,32.6,10.2,4.2,1.1,1.6,-3.3
Luka Doncic,2.9,2.5,DAL,66,PG,0.492,0.761,3.0,32.7,8.8,8.1,1.3,0.5,-3.6
Jayson Tatum,4.7,4.0,BOS,75,"SF,PF",0.466,0.858,3.3,29.5,8.2,4.8,1.1,0.7,-3.0
Tyrese Haliburton,5.9,5.0,IND,73,"PG,SG",0.488,0.857,2.8,23.3,4.0,10.5,1.7,0.4,-2.6
Shai Gilgeous-Alexander,6.1,6.0,OKC,67,"PG,SG",0.486,0.884,1.0,29.6,5.0,5.6,1.5,0.9,-2.9
Stephen Curry,6.7,7.0,GS,65,PG,0.476,0.914,4.7,28.0,5.6,5.6,0.9,0.3,-3.3
Giannis Antetokounmpo,8.7,8.0,MIL,65,"PF,C",0.565,0.683,0.9,28.3,11.6,5.7,1.1,1.0,-3.6
Damian Lillard,10.8,9.0,MIL,67,PG,0.461,0.903,3.8,27.2,4.0,6.9,0.9,0.3,-3.1


Simulate draft

In [41]:
n_teams = 14
roster_size = 3
teams = [[] for _ in range(n_teams)]
for counter, (i, row) in enumerate(df.iloc[0 : roster_size * n_teams].iterrows()):
    odd_round = ((counter // n_teams) % 2 ==0)
    if odd_round:
        corresponding_team = (counter % n_teams)
    else:
        corresponding_team = n_teams - (counter % n_teams) - 1
    teams[corresponding_team].append(row[[
        "ADP", "pick", "TEAM", "GP", "POS", *STAT_COLS
    ]])
teams = [pd.DataFrame(team) for team in teams]

In [42]:
draft_cumu_stats = []
for i, team in enumerate(teams):
    draft_cumu_stats.append((team[STAT_COLS].sum()).T)

In [43]:
[t.index for t in teams]

[Index(['Nikola Jokic', 'Kawhi Leonard', 'Pascal Siakam'], dtype='object'),
 Index(['Joel Embiid', 'Karl-Anthony Towns', 'Paul George'], dtype='object'),
 Index(['Luka Doncic', 'Jimmy Butler', 'Victor Wembanyama'], dtype='object'),
 Index(['Jayson Tatum', 'Lauri Markkanen', 'Dejounte Murray'], dtype='object'),
 Index(['Tyrese Haliburton', 'LeBron James', 'Bam Adebayo'], dtype='object'),
 Index(['Shai Gilgeous-Alexander', 'James Harden', 'Nikola Vucevic'], dtype='object'),
 Index(['Stephen Curry', 'Desmond Bane', 'Jalen Brunson'], dtype='object'),
 Index(['Giannis Antetokounmpo', 'Fred VanVleet', 'Myles Turner'], dtype='object'),
 Index(['Damian Lillard', 'Trae Young', 'Jaylen Brown'], dtype='object'),
 Index(['Kevin Durant', 'Mikal Bridges', 'De'Aaron Fox'], dtype='object'),
 Index(['Anthony Davis', 'Domantas Sabonis', 'Jrue Holiday'], dtype='object'),
 Index(['LaMelo Ball', 'Donovan Mitchell', 'Cade Cunningham'], dtype='object'),
 Index(['Kyrie Irving', 'Jaren Jackson Jr.', 'Darius Ga

In [44]:
pd.DataFrame(draft_cumu_stats, index=np.arange(1, n_teams+1)).rank(ascending=False)

,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
1,2.0,10.0,13.0,10.0,2.0,4.0,3.5,9.0,7.0
2,6.0,6.5,7.0,2.0,3.0,12.0,10.5,5.0,12.0
3,7.0,12.0,12.0,8.0,5.5,9.0,5.0,3.0,3.0
4,10.0,8.0,5.0,5.0,5.5,14.0,6.5,10.0,1.0
5,3.0,11.0,10.0,9.0,8.0,2.0,2.0,8.0,10.0
6,12.0,6.5,11.0,12.0,7.0,5.0,10.5,6.0,11.0
7,8.0,1.0,1.0,4.0,13.0,8.0,12.5,13.0,6.0
8,4.0,13.0,9.0,13.0,4.0,13.0,9.0,1.0,3.0
9,13.0,3.0,3.0,1.0,14.0,1.0,12.5,14.0,14.0
10,5.0,5.0,8.0,3.0,10.0,10.5,14.0,7.0,5.0


In [45]:
import requests

url = "https://hashtagbasketball.com/fantasy-basketball-adp"

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
hashtag_adp = (
    pd.read_html(r.text)[0]
    .drop(columns=["TEAM"])
    .drop_duplicates(keep=False)
    .dropna()
    .astype({
        "YAHOO": "float32",
        "ESPN": "float32",
        "FANTRAX": "float32",
        "BLEND": "float32"
        })
)

In [46]:
sort_by_adp = df.merge(hashtag_adp, left_index=True, right_on="PLAYER").sort_values("BLEND").set_index("PLAYER")

In [47]:
n_teams = 14
roster_size = 2
teams = [[] for _ in range(n_teams)]
for counter, (i, row) in enumerate(sort_by_adp.iloc[0 : roster_size * n_teams].iterrows()):
    odd_round = ((counter // n_teams) % 2 ==0)
    if odd_round:
        corresponding_team = (counter % n_teams)
    else:
        corresponding_team = n_teams - (counter % n_teams) - 1
    teams[corresponding_team].append(row[[
        "ADP", "pick", "TEAM", "GP", "POS", *STAT_COLS
    ]])
teams = [pd.DataFrame(team) for team in teams]

draft_cumu_stats = []
for i, team in enumerate(teams):
    draft_cumu_stats.append((team[STAT_COLS].sum()).T)

In [48]:
[t.index for t in teams]

[Index(['Nikola Jokic', 'Dejounte Murray'], dtype='object'),
 Index(['Luka Doncic', 'Bam Adebayo'], dtype='object'),
 Index(['Joel Embiid', 'Victor Wembanyama'], dtype='object'),
 Index(['Jayson Tatum', 'Jimmy Butler'], dtype='object'),
 Index(['Shai Gilgeous-Alexander', 'Karl-Anthony Towns'], dtype='object'),
 Index(['Giannis Antetokounmpo', 'James Harden'], dtype='object'),
 Index(['Tyrese Haliburton', 'Lauri Markkanen'], dtype='object'),
 Index(['Stephen Curry', 'Jaren Jackson Jr.'], dtype='object'),
 Index(['Anthony Edwards', 'Mikal Bridges'], dtype='object'),
 Index(['Kevin Durant', 'LeBron James'], dtype='object'),
 Index(['Damian Lillard', 'Kyrie Irving'], dtype='object'),
 Index(['Anthony Davis', 'Trae Young'], dtype='object'),
 Index(['LaMelo Ball', 'Donovan Mitchell'], dtype='object'),
 Index(['Devin Booker', 'Domantas Sabonis'], dtype='object')]

In [49]:
pd.DataFrame(draft_cumu_stats, index=np.arange(1, n_teams+1)).rank(ascending=False)

,FG%,FT%,3pm,PTS,TREB,AST,STL,BLK,TO
1,2.0,10.0,12.0,14.0,3.0,1.0,2.0,11.0,9.0
2,3.0,13.0,10.0,2.0,1.0,8.0,5.0,7.0,11.5
3,5.0,11.0,14.0,9.0,2.0,14.0,13.0,1.0,4.0
4,9.0,5.0,7.0,8.0,8.0,11.0,3.5,9.0,2.0
5,7.5,2.0,9.0,7.0,9.0,10.0,8.0,4.0,8.0
6,6.0,14.0,8.0,10.0,5.0,2.0,8.0,5.5,14.0
7,10.0,3.0,4.0,12.0,11.0,6.5,6.0,11.0,1.0
8,11.0,6.0,3.0,11.0,10.0,13.0,12.0,2.0,5.0
9,13.0,9.0,5.0,5.0,13.0,12.0,3.5,8.0,3.0
10,4.0,7.0,6.0,3.0,7.0,9.0,14.0,5.5,11.5
